## Assoc Rules

## 

### Lift measures how likely one is bought given that other is bought, and also taking into account that it is not a fluke / coincidence

In [1]:
import math
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from IPython.display import clear_output

import folium
from folium.plugins import MarkerCluster
from folium import Choropleth, Circle, Marker
from folium import plugins
from folium.plugins import HeatMap

import plotly.offline as py 
import plotly.graph_objects as go
import plotly.express as px
# py.init_notebook_mode(connected=True)

# from matplotlib import rc
# rc('text', usetex=True)

pd.options.display.float_format = '{:,.3f}'.format
pd.set_option('display.max_columns', None)
pd.set_option('max_rows', 200)
pd.set_option('display.max_colwidth', 150)

%matplotlib inline

In [2]:
data_encoded = pd.read_csv("data_encoded.csv")

In [3]:
phone_serivces = ['PhoneService','MultipleLines']
internet_services = ['InternetService','InternetType','UnlimitedData', 
                            'StreamingTV', 'StreamingMovies', 'StreamingMusic'
                             'OnlineSecurity','OnlineBackup',
                            'DeviceProtectionPlan','PremiumTechSupport']
other_services = ['PaperlessBilling','Contract']

internet_services_encoded=['InternetService','UnlimitedData', 'InternetType_NA',
                            'InternetType_DSL', 'InternetType_Cable', 'InternetType_Fiber Optic',
                            'StreamingTV', 'StreamingMovies', 'StreamingMusic',
                             'OnlineSecurity','OnlineBackup',
                            'DeviceProtectionPlan','PremiumTechSupport']

other_services_encoded=['PaperlessBilling',
                       'Contract_Two year','Contract_One year','Contract_Month-to-month']

# omitted variables: Offer, PaymentMethod
# multicat = ['Offer', 'PaymentMethod', 'InternetType', 'Contract']

all_services = phone_serivces+internet_services+other_services
all_services_encoded = phone_serivces+internet_services_encoded+other_services_encoded

In [4]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

min_sup=0.4
min_thres=0.4

print("Settings: ")
print("min_support: ", min_sup)
print("min_thres: ", min_thres)

Settings: 
min_support:  0.4
min_thres:  0.4


In [5]:
# Assoc Rule for all services
assoc_rule_data_all = data_encoded[all_services_encoded]
assoc_rule_data_all = assoc_rule_data_all.replace('No internet service',0)
assoc_rule_data_all = assoc_rule_data_all.replace('No phone service',0)

assoc_rule_data_all = assoc_rule_data_all.replace({"1":1, "0":0})

item_all = apriori(assoc_rule_data_all, use_colnames=True, min_support=min_sup)
assoc_rule_all = association_rules(item_all, metric = 'confidence', min_threshold=min_thres)

# Assoc Rule for Internet service
assoc_rule_data_internet = data_encoded[data_encoded['InternetService']==1][internet_services_encoded]
assoc_rule_data_internet = assoc_rule_data_internet.drop('InternetType_NA',axis=1)

assoc_rule_data_internet = assoc_rule_data_internet.replace({"1":1, "0":0})

item_internet = apriori(assoc_rule_data_internet, use_colnames=True, min_support=min_sup)
assoc_rule_internet = association_rules(item_internet, metric = 'confidence', min_threshold=min_thres)

# Assoc Rule for Internet service + others (paperless billing & contract)
assoc_rule_data_internet_others = data_encoded[data_encoded['InternetService']==1][internet_services_encoded+other_services_encoded]
assoc_rule_data_internet_others = assoc_rule_data_internet_others.drop('InternetType_NA',axis=1)

assoc_rule_data_internet_others = assoc_rule_data_internet_others.replace({"1":1, "0":0})

item_internet_others = apriori(assoc_rule_data_internet_others, use_colnames=True, min_support=min_sup)
assoc_rule_internet_others = association_rules(item_internet_others, metric = 'confidence', min_threshold=min_thres)

# Assoc Rule for phone service + others (paperless billing & contract)
assoc_rule_data_phone_others = data_encoded[data_encoded['PhoneService']==1][phone_serivces+other_services_encoded]

assoc_rule_data_phone_others = assoc_rule_data_phone_others.replace({"1":1, "0":0})

item_phone_others = apriori(assoc_rule_data_phone_others, use_colnames=True, min_support=min_sup)
assoc_rule_phone_others = association_rules(item_phone_others, metric = 'confidence', min_threshold=min_thres)

In [6]:
for i in range(0,len(assoc_rule_data_all.columns)):
    print(assoc_rule_data_all.columns[i],"\n",assoc_rule_data_all[assoc_rule_data_all.columns[i]].unique())

PhoneService 
 [0 1]
MultipleLines 
 [0 1]
InternetService 
 [1 0]
UnlimitedData 
 [1 0]
InternetType_NA 
 [0 1]
InternetType_DSL 
 [0 1]
InternetType_Cable 
 [0 1]
InternetType_Fiber Optic 
 [1 0]
StreamingTV 
 [0 1]
StreamingMovies 
 [0 1]
StreamingMusic 
 [0 1]
OnlineSecurity 
 [0 1]
OnlineBackup 
 [1 0]
DeviceProtectionPlan 
 [0 1]
PremiumTechSupport 
 [0 1]
PaperlessBilling 
 [1 0]
Contract_Two year 
 [0 1]
Contract_One year 
 [0 1]
Contract_Month-to-month 
 [1 0]


In [7]:
assoc_rule_data_all = assoc_rule_data_all.replace({"1":1, "0":0})

In [8]:
assoc_rule_data_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype
---  ------                    --------------  -----
 0   PhoneService              7043 non-null   int64
 1   MultipleLines             7043 non-null   int64
 2   InternetService           7043 non-null   int64
 3   UnlimitedData             7043 non-null   int64
 4   InternetType_NA           7043 non-null   int64
 5   InternetType_DSL          7043 non-null   int64
 6   InternetType_Cable        7043 non-null   int64
 7   InternetType_Fiber Optic  7043 non-null   int64
 8   StreamingTV               7043 non-null   int64
 9   StreamingMovies           7043 non-null   int64
 10  StreamingMusic            7043 non-null   int64
 11  OnlineSecurity            7043 non-null   int64
 12  OnlineBackup              7043 non-null   int64
 13  DeviceProtectionPlan      7043 non-null   int64
 14  PremiumTechSupport        7043 non-null 

In [9]:
## Results DF Analysis - based on lift 
assoc_rule_all = assoc_rule_all.sort_values("lift",ascending=False)
assoc_rule_internet  = assoc_rule_internet .sort_values("lift",ascending=False)
assoc_rule_internet_others  = assoc_rule_internet_others .sort_values("lift",ascending=False)
assoc_rule_phone_others  = assoc_rule_phone_others .sort_values("lift",ascending=False)

### Association Rules created based on the whole dataset

In [10]:
assoc_rule_all

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
12,"(InternetService, PhoneService)",(PaperlessBilling),0.686,0.592,0.474,0.690,1.166,0.067,1.317
17,(PaperlessBilling),"(InternetService, PhoneService)",0.592,0.686,0.474,0.800,1.166,0.067,1.570
8,(InternetService),(PaperlessBilling),0.783,0.592,0.529,0.675,1.140,0.065,1.255
9,(PaperlessBilling),(InternetService),0.592,0.783,0.529,0.893,1.140,0.065,2.026
15,(InternetService),"(PhoneService, PaperlessBilling)",0.783,0.537,0.474,0.605,1.126,0.053,1.172
14,"(PhoneService, PaperlessBilling)",(InternetService),0.537,0.783,0.474,0.882,1.126,0.053,1.838
18,"(InternetService, PhoneService)",(Contract_Month-to-month),0.686,0.550,0.422,0.615,1.118,0.045,1.169
23,(Contract_Month-to-month),"(InternetService, PhoneService)",0.550,0.686,0.422,0.768,1.118,0.045,1.350
1,(MultipleLines),(PhoneService),0.422,0.903,0.422,1.000,1.107,0.041,inf
0,(PhoneService),(MultipleLines),0.903,0.422,0.422,0.467,1.107,0.041,1.085


In [11]:
assoc_rule_internet

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
21,(StreamingMusic),"(InternetService, StreamingMovies)",0.451,0.495,0.427,0.947,1.913,0.204,9.587
20,(StreamingMovies),"(InternetService, StreamingMusic)",0.495,0.451,0.427,0.863,1.913,0.204,4.000
17,"(InternetService, StreamingMusic)",(StreamingMovies),0.451,0.495,0.427,0.947,1.913,0.204,9.587
16,"(InternetService, StreamingMovies)",(StreamingMusic),0.495,0.451,0.427,0.863,1.913,0.204,4.000
15,(StreamingMusic),(StreamingMovies),0.451,0.495,0.427,0.947,1.913,0.204,9.587
14,(StreamingMovies),(StreamingMusic),0.495,0.451,0.427,0.863,1.913,0.204,4.000
1,(UnlimitedData),(InternetService),0.490,1.000,0.490,1.000,1.000,0.000,inf
19,(InternetService),"(StreamingMovies, StreamingMusic)",1.000,0.427,0.427,0.427,1.000,0.000,1.000
18,"(StreamingMovies, StreamingMusic)",(InternetService),0.427,1.000,0.427,1.000,1.000,0.000,inf
13,(DeviceProtectionPlan),(InternetService),0.439,1.000,0.439,1.000,1.000,0.000,inf


In [12]:
assoc_rule_internet_others

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
26,(StreamingMovies),"(InternetService, StreamingMusic)",0.495,0.451,0.427,0.863,1.913,0.204,4.000
27,(StreamingMusic),"(InternetService, StreamingMovies)",0.451,0.495,0.427,0.947,1.913,0.204,9.587
19,(StreamingMusic),(StreamingMovies),0.451,0.495,0.427,0.947,1.913,0.204,9.587
22,"(InternetService, StreamingMovies)",(StreamingMusic),0.495,0.451,0.427,0.863,1.913,0.204,4.000
23,"(InternetService, StreamingMusic)",(StreamingMovies),0.451,0.495,0.427,0.947,1.913,0.204,9.587
18,(StreamingMovies),(StreamingMusic),0.495,0.451,0.427,0.863,1.913,0.204,4.000
29,"(InternetService, PaperlessBilling)",(Contract_Month-to-month),0.675,0.607,0.437,0.647,1.066,0.027,1.113
21,(PaperlessBilling),(Contract_Month-to-month),0.675,0.607,0.437,0.647,1.066,0.027,1.113
33,(PaperlessBilling),"(InternetService, Contract_Month-to-month)",0.675,0.607,0.437,0.647,1.066,0.027,1.113
28,"(InternetService, Contract_Month-to-month)",(PaperlessBilling),0.607,0.675,0.437,0.719,1.066,0.027,1.158


In [13]:
assoc_rule_phone_others

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(PhoneService),(MultipleLines),1.000,0.467,0.467,0.467,1.000,0.000,1.000
1,(MultipleLines),(PhoneService),0.467,1.000,0.467,1.000,1.000,0.000,inf
2,(PhoneService),(PaperlessBilling),1.000,0.595,0.595,0.595,1.000,0.000,1.000
3,(PaperlessBilling),(PhoneService),0.595,1.000,0.595,1.000,1.000,0.000,inf
4,(PhoneService),(Contract_Month-to-month),1.000,0.550,0.550,0.550,1.000,0.000,1.000
5,(Contract_Month-to-month),(PhoneService),0.550,1.000,0.550,1.000,1.000,0.000,inf


### Assoc Rules based on Churn == 1

In [14]:
data_churn = pd.DataFrame()
data_churn = data_encoded[data_encoded["Churn"] == 1]

In [15]:
# Assoc Rule for all services
assoc_rule_data_all = data_churn[all_services_encoded]
assoc_rule_data_all = assoc_rule_data_all.replace('No internet service',0)
assoc_rule_data_all = assoc_rule_data_all.replace('No phone service',0)
assoc_rule_data_all = assoc_rule_data_all.replace({"1":1, "0":0})

item_all = apriori(assoc_rule_data_all, use_colnames=True, min_support=min_sup)
assoc_rule_all = association_rules(item_all, metric = 'confidence', min_threshold=min_thres)

# Assoc Rule for Internet service
assoc_rule_data_internet = data_churn[data_encoded['InternetService']==1][internet_services_encoded]
assoc_rule_data_internet = assoc_rule_data_internet.drop('InternetType_NA',axis=1)
assoc_rule_data_internet = assoc_rule_data_internet.replace({"1":1, "0":0})

item_internet = apriori(assoc_rule_data_internet, use_colnames=True, min_support=min_sup)
assoc_rule_internet = association_rules(item_internet, metric = 'confidence', min_threshold=min_thres)

# Assoc Rule for Internet service + others (paperless billing & contract)
assoc_rule_data_internet_others = data_churn[data_encoded['InternetService']==1][internet_services_encoded+other_services_encoded]
assoc_rule_data_internet_others = assoc_rule_data_internet_others.drop('InternetType_NA',axis=1)
assoc_rule_data_internet_others = assoc_rule_data_internet_others.replace({"1":1, "0":0})

item_internet_others = apriori(assoc_rule_data_internet_others, use_colnames=True, min_support=min_sup)
assoc_rule_internet_others = association_rules(item_internet_others, metric = 'confidence', min_threshold=min_thres)

# Assoc Rule for phone service + others (paperless billing & contract)
assoc_rule_data_phone_others = data_churn[data_encoded['PhoneService']==1][phone_serivces+other_services_encoded]
assoc_rule_data_phone_others = assoc_rule_data_phone_others.replace({"1":1, "0":0})

item_phone_others = apriori(assoc_rule_data_phone_others, use_colnames=True, min_support=min_sup)
assoc_rule_phone_others = association_rules(item_phone_others, metric = 'confidence', min_threshold=min_thres)

## Results DF Analysis - based on lift 
assoc_rule_all = assoc_rule_all.sort_values("lift",ascending=False)
assoc_rule_internet  = assoc_rule_internet .sort_values("lift",ascending=False)
assoc_rule_internet_others  = assoc_rule_internet_others .sort_values("lift",ascending=False)
assoc_rule_phone_others  = assoc_rule_phone_others .sort_values("lift",ascending=False)

C:\Users\hp\anaconda3\lib\site-packages\ipykernel_launcher.py:11: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\hp\anaconda3\lib\site-packages\ipykernel_launcher.py:19: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\hp\anaconda3\lib\site-packages\ipykernel_launcher.py:27: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [16]:
assoc_rule_all

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
29,(MultipleLines),"(InternetService, PhoneService)",0.455,0.849,0.449,0.988,1.165,0.064,12.871
24,"(InternetService, PhoneService)",(MultipleLines),0.849,0.455,0.449,0.530,1.165,0.064,1.159
0,(PhoneService),(MultipleLines),0.909,0.455,0.455,0.500,1.100,0.041,1.091
28,(PhoneService),"(InternetService, MultipleLines)",0.909,0.449,0.449,0.494,1.100,0.041,1.089
25,"(InternetService, MultipleLines)",(PhoneService),0.449,0.909,0.449,1.000,1.100,0.041,inf
1,(MultipleLines),(PhoneService),0.455,0.909,0.455,1.000,1.100,0.041,inf
30,"(InternetService, PhoneService)",(StreamingTV),0.849,0.436,0.401,0.473,1.086,0.032,1.071
35,(StreamingTV),"(InternetService, PhoneService)",0.436,0.849,0.401,0.921,1.086,0.032,1.926
33,(InternetService),"(PhoneService, StreamingTV)",0.940,0.401,0.401,0.427,1.064,0.024,1.045
32,"(PhoneService, StreamingTV)",(InternetService),0.401,0.940,0.401,1.000,1.064,0.024,inf


In [17]:
assoc_rule_internet

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(InternetService),(InternetType_Cable),1.000,0.454,0.454,0.454,1.000,0.000,1.000
1,(InternetType_Cable),(InternetService),0.454,1.000,0.454,1.000,1.000,0.000,inf
2,(InternetService),(StreamingTV),1.000,0.464,0.464,0.464,1.000,0.000,1.000
3,(StreamingTV),(InternetService),0.464,1.000,0.464,1.000,1.000,0.000,inf
4,(InternetService),(StreamingMovies),1.000,0.466,0.466,0.466,1.000,0.000,1.000
5,(StreamingMovies),(InternetService),0.466,1.000,0.466,1.000,1.000,0.000,inf
6,(InternetService),(StreamingMusic),1.000,0.415,0.415,0.415,1.000,0.000,1.000
7,(StreamingMusic),(InternetService),0.415,1.000,0.415,1.000,1.000,0.000,inf


In [18]:
assoc_rule_internet_others

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
14,(Contract_Month-to-month),(PaperlessBilling),0.886,0.773,0.690,0.778,1.006,0.004,1.022
15,(PaperlessBilling),(Contract_Month-to-month),0.773,0.886,0.690,0.892,1.006,0.004,1.052
26,(Contract_Month-to-month),"(InternetService, PaperlessBilling)",0.886,0.773,0.690,0.778,1.006,0.004,1.022
23,"(InternetService, PaperlessBilling)",(Contract_Month-to-month),0.773,0.886,0.690,0.892,1.006,0.004,1.052
22,"(InternetService, Contract_Month-to-month)",(PaperlessBilling),0.886,0.773,0.690,0.778,1.006,0.004,1.022
27,(PaperlessBilling),"(InternetService, Contract_Month-to-month)",0.773,0.886,0.690,0.892,1.006,0.004,1.052
17,"(InternetService, Contract_Month-to-month)",(InternetType_Cable),0.886,0.454,0.404,0.456,1.003,0.001,1.002
13,(Contract_Month-to-month),(InternetType_Cable),0.886,0.454,0.404,0.456,1.003,0.001,1.002
21,(Contract_Month-to-month),"(InternetService, InternetType_Cable)",0.886,0.454,0.404,0.456,1.003,0.001,1.002
12,(InternetType_Cable),(Contract_Month-to-month),0.454,0.886,0.404,0.888,1.003,0.001,1.021


In [19]:
assoc_rule_phone_others

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
16,(MultipleLines),"(PhoneService, PaperlessBilling)",0.500,0.755,0.416,0.832,1.102,0.039,1.459
6,(MultipleLines),(PaperlessBilling),0.500,0.755,0.416,0.832,1.102,0.039,1.459
7,(PaperlessBilling),(MultipleLines),0.755,0.500,0.416,0.551,1.102,0.039,1.114
12,"(PhoneService, MultipleLines)",(PaperlessBilling),0.500,0.755,0.416,0.832,1.102,0.039,1.459
13,"(PhoneService, PaperlessBilling)",(MultipleLines),0.755,0.500,0.416,0.551,1.102,0.039,1.114
17,(PaperlessBilling),"(PhoneService, MultipleLines)",0.755,0.500,0.416,0.551,1.102,0.039,1.114
29,(PaperlessBilling),"(PhoneService, Contract_Month-to-month)",0.755,0.884,0.672,0.891,1.008,0.005,1.062
28,(Contract_Month-to-month),"(PhoneService, PaperlessBilling)",0.884,0.755,0.672,0.760,1.008,0.005,1.024
25,"(PhoneService, PaperlessBilling)",(Contract_Month-to-month),0.755,0.884,0.672,0.891,1.008,0.005,1.062
24,"(PhoneService, Contract_Month-to-month)",(PaperlessBilling),0.884,0.755,0.672,0.760,1.008,0.005,1.024


## Assoc rules based on data where Churn == 0

In [20]:
data_not_churn = pd.DataFrame()
data_not_churn = data_encoded[data_encoded["Churn"] == 0]

In [21]:
# Assoc Rule for all services
assoc_rule_data_all = data_not_churn[all_services_encoded]
assoc_rule_data_all = assoc_rule_data_all.replace('No internet service',0)
assoc_rule_data_all = assoc_rule_data_all.replace('No phone service',0)
assoc_rule_data_all = assoc_rule_data_all.replace({"1":1, "0":0})

item_all = apriori(assoc_rule_data_all, use_colnames=True, min_support=min_sup)
assoc_rule_all = association_rules(item_all, metric = 'confidence', min_threshold=min_thres)

# Assoc Rule for Internet service
assoc_rule_data_internet = data_not_churn[data_encoded['InternetService']==1][internet_services_encoded]
assoc_rule_data_internet = assoc_rule_data_internet.drop('InternetType_NA',axis=1)
assoc_rule_data_internet = assoc_rule_data_internet.replace({"1":1, "0":0})

item_internet = apriori(assoc_rule_data_internet, use_colnames=True, min_support=min_sup)
assoc_rule_internet = association_rules(item_internet, metric = 'confidence', min_threshold=min_thres)

# Assoc Rule for Internet service + others (paperless billing & contract)
assoc_rule_data_internet_others = data_not_churn[data_encoded['InternetService']==1][internet_services_encoded+other_services_encoded]
assoc_rule_data_internet_others = assoc_rule_data_internet_others.drop('InternetType_NA',axis=1)
assoc_rule_data_internet_others = assoc_rule_data_internet_others.replace({"1":1, "0":0})

item_internet_others = apriori(assoc_rule_data_internet_others, use_colnames=True, min_support=min_sup)
assoc_rule_internet_others = association_rules(item_internet_others, metric = 'confidence', min_threshold=min_thres)

# Assoc Rule for phone service + others (paperless billing & contract)
assoc_rule_data_phone_others = data_not_churn[data_encoded['PhoneService']==1][phone_serivces+other_services_encoded]
assoc_rule_data_phone_others = assoc_rule_data_phone_others.replace({"1":1, "0":0})

item_phone_others = apriori(assoc_rule_data_phone_others, use_colnames=True, min_support=min_sup)
assoc_rule_phone_others = association_rules(item_phone_others, metric = 'confidence', min_threshold=min_thres)

## Results DF Analysis - based on lift 
assoc_rule_all = assoc_rule_all.sort_values("lift",ascending=False)
assoc_rule_internet  = assoc_rule_internet .sort_values("lift",ascending=False)
assoc_rule_internet_others  = assoc_rule_internet_others .sort_values("lift",ascending=False)
assoc_rule_phone_others  = assoc_rule_phone_others .sort_values("lift",ascending=False)

C:\Users\hp\anaconda3\lib\site-packages\ipykernel_launcher.py:11: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\hp\anaconda3\lib\site-packages\ipykernel_launcher.py:19: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\hp\anaconda3\lib\site-packages\ipykernel_launcher.py:27: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [22]:
assoc_rule_all

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
6,(InternetService),(UnlimitedData),0.727,0.426,0.426,0.586,1.376,0.116,1.387
7,(UnlimitedData),(InternetService),0.426,0.727,0.426,1.000,1.376,0.116,inf
10,"(InternetService, PhoneService)",(PaperlessBilling),0.628,0.536,0.405,0.646,1.206,0.069,1.311
15,(PaperlessBilling),"(InternetService, PhoneService)",0.536,0.628,0.405,0.757,1.206,0.069,1.532
8,(InternetService),(PaperlessBilling),0.727,0.536,0.457,0.629,1.175,0.068,1.253
9,(PaperlessBilling),(InternetService),0.536,0.727,0.457,0.854,1.175,0.068,1.873
12,"(PhoneService, PaperlessBilling)",(InternetService),0.484,0.727,0.405,0.839,1.154,0.054,1.691
13,(InternetService),"(PhoneService, PaperlessBilling)",0.727,0.484,0.405,0.558,1.154,0.054,1.168
0,(PhoneService),(MultipleLines),0.901,0.410,0.410,0.455,1.110,0.041,1.083
1,(MultipleLines),(PhoneService),0.410,0.901,0.410,1.000,1.110,0.041,inf


In [23]:
assoc_rule_internet

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
25,(StreamingMusic),"(InternetService, StreamingMovies)",0.468,0.509,0.456,0.975,1.916,0.218,19.644
24,(StreamingMovies),"(InternetService, StreamingMusic)",0.509,0.468,0.456,0.897,1.916,0.218,5.143
21,"(InternetService, StreamingMusic)",(StreamingMovies),0.468,0.509,0.456,0.975,1.916,0.218,19.644
20,"(InternetService, StreamingMovies)",(StreamingMusic),0.509,0.468,0.456,0.897,1.916,0.218,5.143
19,(StreamingMusic),(StreamingMovies),0.468,0.509,0.456,0.975,1.916,0.218,19.644
18,(StreamingMovies),(StreamingMusic),0.509,0.468,0.456,0.897,1.916,0.218,5.143
1,(UnlimitedData),(InternetService),0.586,1.000,0.586,1.000,1.000,0.000,inf
23,(InternetService),"(StreamingMovies, StreamingMusic)",1.000,0.456,0.456,0.456,1.000,0.000,1.000
22,"(StreamingMovies, StreamingMusic)",(InternetService),0.456,1.000,0.456,1.000,1.000,0.000,inf
17,(PremiumTechSupport),(InternetService),0.461,1.000,0.461,1.000,1.000,0.000,inf


In [24]:
assoc_rule_internet_others

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
29,(StreamingMusic),"(InternetService, StreamingMovies)",0.468,0.509,0.456,0.975,1.916,0.218,19.644
28,(StreamingMovies),"(InternetService, StreamingMusic)",0.509,0.468,0.456,0.897,1.916,0.218,5.143
25,"(InternetService, StreamingMusic)",(StreamingMovies),0.468,0.509,0.456,0.975,1.916,0.218,19.644
24,"(InternetService, StreamingMovies)",(StreamingMusic),0.509,0.468,0.456,0.897,1.916,0.218,5.143
23,(StreamingMusic),(StreamingMovies),0.468,0.509,0.456,0.975,1.916,0.218,19.644
22,(StreamingMovies),(StreamingMusic),0.509,0.468,0.456,0.897,1.916,0.218,5.143
1,(UnlimitedData),(InternetService),0.586,1.000,0.586,1.000,1.000,0.000,inf
27,(InternetService),"(StreamingMovies, StreamingMusic)",1.000,0.456,0.456,0.456,1.000,0.000,1.000
26,"(StreamingMovies, StreamingMusic)",(InternetService),0.456,1.000,0.456,1.000,1.000,0.000,inf
21,(Contract_Month-to-month),(InternetService),0.477,1.000,0.477,1.000,1.000,0.000,inf


In [25]:
assoc_rule_phone_others

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(PhoneService),(MultipleLines),1.000,0.455,0.455,0.455,1.000,0.000,1.000
1,(MultipleLines),(PhoneService),0.455,1.000,0.455,1.000,1.000,0.000,inf
2,(PhoneService),(PaperlessBilling),1.000,0.537,0.537,0.537,1.000,0.000,1.000
3,(PaperlessBilling),(PhoneService),0.537,1.000,0.537,1.000,1.000,0.000,inf
4,(PhoneService),(Contract_Month-to-month),1.000,0.428,0.428,0.428,1.000,0.000,1.000
5,(Contract_Month-to-month),(PhoneService),0.428,1.000,0.428,1.000,1.000,0.000,inf
